In [3]:
import cv2 as cv
import numpy as np
from PIL import Image
import scipy
import scipy.misc
import scipy.cluster
from numba import njit, prange
from timeit import default_timer as timer

In [4]:
START_POINT = (0, 0)
END_POINT = (150, 150)

vid_width = 0
vid_height = 0

RECTANGLE_COLOR = (255, 0, 0)
FONT_COLOR = (124, 252, 0)
FONT_SCALE = 1
THICKNESS = 1
LINE_TYPE = 2
TEXT_LEFT_BOTTOM_CORNER = (0, 0)

COMMAND_KEYS = {
    2490368: (0, -10),  # UpKey
    2621440: (0, 10),  # DownKey
    2424832: (-10, 0),  # LeftKey
    2555904: (10, 0)  # RightKey
}

COLOR_SCHEME = {
    "Red": ([0, 70, 50], [10, 255, 255]),
    "Green": ([40, 40, 40], [70, 255, 255]),
    "Blue": ([100, 150, 0], [140, 255, 255]),
    "Yellow": ([25, 50, 50], [32, 255, 255])
}

COLOR_SCHEME_LIST = [
    ["Red", ([0, 70, 50], [10, 255, 255])],
    ["Green", ([40, 40, 40], [70, 255, 255])],
    ["Blue", ([100, 150, 0], [140, 255, 255])],
    ["Yellow", ([25, 50, 50], [32, 255, 255])]
]

def process(img, pr_type=1):

    def _get_color_by_np(img):
        image = img[START_POINT[1]:END_POINT[1], START_POINT[0]:END_POINT[0]].copy()
        hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)

        color_name = None
        max_mask_size = 0

        # Сравниваем размер маски для каждого цвета
        for cur_name, (lower, upper) in COLOR_SCHEME.items():
#             mask = cv.inRange(hsv, lower, upper)
            mask_size = ((lower < hsv) & (hsv < upper)).sum()
            if mask_size > max_mask_size:
                max_mask_size = mask_size
                color_name = cur_name
        
        return color_name
    
    def _get_color_by_pil(img):
        img = img[START_POINT[1]:END_POINT[1], START_POINT[0]:END_POINT[0]].copy()
        pil_img = Image.fromarray(img)
        image = pil_img.copy()
        image = image.convert("HSV")
        image = image.resize((1, 1), resample=0)
        dominant_color = list(image.getpixel((0, 0)))
        for cur_name, (lower, upper) in COLOR_SCHEME.items():
            if (lower < dominant_color) and (dominant_color < upper):
                return cur_name

    def _get_color_by_np_njit(img):
        @njit
        def _get_color(hsv):
            color_name = None
            max_mask_size = 0
            # Сравниваем размер маски для каждого цвета
            for cur_color in COLOR_SCHEME_LIST:
                cur_name = cur_color[0]
                (lower, upper) = cur_color[1]
                mask_size = ((lower < hsv) & (hsv < upper)).sum()
                if mask_size > max_mask_size:
                    max_mask_size = mask_size
                    color_name = cur_name
            return color_name
        image = img[START_POINT[1]:END_POINT[1], START_POINT[0]:END_POINT[0]].copy()
        hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
        return _get_color(hsv)
        
    GET_COLOR_FUNCS = {
        1: _get_color_by_np,
        2: _get_color_by_pil,
        3: _get_color_by_np_njit
    }

    com = GET_COLOR_FUNCS.get(pr_type)
    color_name = com(img) or "Not found"
    
    # Область в которой мы определяем цвет
    cv.rectangle(img, START_POINT, END_POINT, RECTANGLE_COLOR, LINE_TYPE, THICKNESS*8)

    # Отображение названия цвета
    cv.rectangle(img, (vid_width-160, vid_height-50), (vid_width, vid_height), (0, 0, 0), -1)
    cv.putText(img, color_name,
               TEXT_LEFT_BOTTOM_CORNER,
               cv.FONT_HERSHEY_SIMPLEX,
               FONT_SCALE,
               FONT_COLOR,
               THICKNESS,
               LINE_TYPE)

    return img

In [9]:
capture = cv.VideoCapture("C:/Users/whr1t/Desktop/Repos/CV/videoplayback.mp4")
# capture = cv.VideoCapture(0)

vid_width = int(capture.get(cv.CAP_PROP_FRAME_WIDTH))
vid_height = int(capture.get(cv.CAP_PROP_FRAME_HEIGHT))

TEXT_LEFT_BOTTOM_CORNER = (vid_width - 160, vid_height - 10)

avg_time = np.array([])

while True:
    ret, frame = capture.read()
    if ret is True:
        start = timer()
        result = process(frame, 1)
        end = timer()
        avg_time = np.append(avg_time, end - start)
        cv.imshow("result", result)

        k = cv.waitKeyEx(100)
        frame_move = COMMAND_KEYS.get(k)
        if frame_move:
            START_POINT = tuple(map(lambda x, y: x + y, START_POINT, frame_move))
            END_POINT = tuple(map(lambda x, y: x + y, END_POINT, frame_move))

        if k == 27:  # ESC
            break
    else:
        break

print(np.mean(avg_time))
cv.waitKey(0)
capture.release()
cv.destroyAllWindows()

0.0028085147727510544
